In [66]:
import matplotlib.pyplot as plt
import math
import numpy as np
import os
np.set_printoptions(precision=17)

import importlib
import utils
importlib.reload(utils);

In [67]:
SAVE_LOC = 'new_names'
H_LOC = "h_removable/h{:}"
HP_LOC = "hp_removable/h{:}p"
TORCH_LOC = "hp_removable_torch/h{:}p"
TF_LOC = "hp_removable_tensorflow/h{:}p"
LOCS = [H_LOC,HP_LOC,TORCH_LOC,TF_LOC]

#FIG_WIDTH = utils.set_size("autodiff",fraction=1,subplots=(1,1))
utils.create_folders([os.path.join(SAVE_LOC,l) for l in LOCS], ['','1','2','3','4'])


# Functions $f$, $g$, $h$, $h'$, $h_i$, and $h_i'$.  

In [68]:
def f(x):
    return x**2 - 4

def fp(x):
    return 2*x 

def g(x):
    return x-2

def gp(x):
    return 1

def sign(x,case_zero=0):
    if x > 0:
        return 1
    if x < 0:
        return -1
    if x == 0:
        return case_zero
    
def sign_p(x,case_zero=0):
    return 0
    
def sigmoid(x,k):
    return 2/(1+math.exp(-k*x))-1

def sigmoid_p(x,k):
    return (2*k*math.exp(-k*x))/((1+math.exp(-k*x))**2)
    
def h(x,e,d,sp,f,g,*args):
    return f(x) / g(x)

def h_under(gamma,e,d,sp,f,g,*args):
    num_underflow = f(2+gamma) == ((2+gamma)**2 - gamma**2)-4 and gamma != 0
    den_underflow = g(2+gamma) == 0 and gamma != 0
    return num_underflow, den_underflow

def hp_q(x,e,d,sigmoid_p,f,g,fp,gp,*args):
    return (fp(x)*g(x) - f(x)*gp(x))/(g(x)**2)

def hp_p(x,e,d,sp,f,g,fp,gp,*args):
    return fp(x)/g(x) - f(x)*gp(x)/(g(x)**2) 

# 2x(x-2) - (x^2 - 4)
# 2(2 + gamma)(2 + gamma - 2) - ((2 + gamma)^2 - 4)
# x^2 - 4x + 4 = (x-2)^2 
# gamma^2

def hp_under(gamma,e,d,cz,f,g,fp,gp,*args):
    # TODO: Expand out numerator and figure out which term drops first.
    num_underflow = fp(2+gamma) != gamma**2 #and gamma != 0 
    den_underflow = g(2+gamma)**2 == 0 and gamma != 0
    print("{:3e} {:4} {:4} {:15e} {:15e} {:15e} ".format(gamma, num_underflow, den_underflow, fp(2+gamma)*g(2+gamma) - f(2+gamma)*gp(2+gamma), g(2+gamma)**2,gamma**2))
    return num_underflow, den_underflow

#######################
######### h1 ##########
#######################

def h1(x,epsilon,delta,sign_param,f,g,sign=sign,*args):
    fx = f(x)
    gx = g(x)
    if 2-delta < x < 2 + delta:
        return fx/(gx + sign(gx,sign_param)*epsilon)
    return fx/gx

def h1p_q(x,epsilon,delta,sign_param,f,g,fp,gp,S,Sp):
    fx = f(x)
    gx = g(x)
    fpx = fp(x)
    gpx = gp(x)
    if 2-delta < x < 2+delta:
        denom = (gx + S(gx,sign_param)*epsilon)
        return (fpx*denom - fx*gpx*(1+Sp(gx,sign_param)*epsilon))/(denom**2)
    return (fpx*gx - fx*gpx)/(gx**2)

def h1p_p(x,epsilon,delta,sign_param,f,g,fp,gp,S,Sp):
    fx = f(x)
    gx = g(x)
    fpx = fp(x)
    gpx = gp(x)
    if 2-delta < x < 2+delta:
        denom = (gx + S(gx,sign_param)*epsilon)
        return fpx/denom - (fx*gpx*(1+Sp(gx,sign_param)*epsilon))/(denom**2)
    return fpx/gx - fx*gpx/(gx**2) 

#######################
######### h2 ##########
#######################

def h2(x,epsilon,delta,sign_param,f,g,sign=sign,*args):
    fx = f(x)
    gx = g(x)
    if 2 - delta < x < 2 + delta:
        return fx/(sign(gx,sign_param)*epsilon)
    return fx/gx

def h2p_q(x,epsilon,delta,sign_param,f,g,fp,gp,S,Sp):
    fx = f(x)
    gx = g(x)
    fpx = fp(x)
    gpx = gp(x)
    if 2 - delta < x < 2 + delta:
        denom = (S(gx,sign_param)*epsilon)
        return (fpx*denom - fx*Sp(gx,sign_param)*gpx*epsilon)/(denom**2)
    return (fpx*gx - fx*gpx)/(gx**2)

def h2p_p(x,epsilon,delta,sign_param,f,g,fp,gp,S,Sp):
    fx = f(x)
    gx = g(x)
    fpx = fp(x)
    gpx = gp(x)
    if 2 - delta < x < 2 + delta:
        denom = (S(gx,sign_param)*epsilon)
        return fpx/denom - (fx*Sp(gx,sign_param)*gpx*epsilon)/(denom**2)
    return fpx/gx - fx*gpx/(gx**2)

#######################
######### h3 ##########
#######################

def h3(x,epsilon,delta,sign_param,f,g,sign=sign,*args):
    fx = f(x)
    gx = g(x)
    if - epsilon < gx < epsilon:
        return fx/(sign(gx,sign_param)*epsilon)
    return fx/gx

def h3p_q(x,epsilon,delta,sign_param,f,g,fp,gp,S,Sp):
    fx = f(x)
    gx = g(x)
    fpx = fp(x)
    gpx = gp(x)
    if - epsilon < gx < epsilon:
        denom = (S(gx,sign_param)*epsilon)
        return (fpx*denom - fx*Sp(gx,sign_param)*gpx*epsilon)/(denom**2) 
    return (fpx*gx - fx*gpx)/(gx**2)

def h3p_p(x,epsilon,delta,sign_param,f,g,fp,gp,S,Sp):
    fx = f(x)
    gx = g(x)
    fpx = fp(x)
    gpx = gp(x)
    if - epsilon < gx < epsilon:
        denom = (S(gx,sign_param)*epsilon)
        return fpx/denom - (fx*Sp(gx,sign_param)*gpx*epsilon)/(denom**2) 
    return fpx/gx - fx*gpx/(gx**2)

#######################
######### h4 ##########
#######################

def h4(x,epsilon,delta,sign_param,f,g,sign=sign,*args):
    fx = f(x)
    gx = g(x)
    if - epsilon < gx < 0:
        return fx/(-epsilon)
    elif 0 <= gx < epsilon:
        return fx/epsilon
    return fx/gx

def h4p_q(x,epsilon,delta,sign_param,f,g,fp,gp,S,Sp):
    fx = f(x)
    gx = g(x)
    fpx = fp(x)
    gpx = gp(x)
    if - epsilon < gx < 0:
        return fpx/(-epsilon)
    elif 0 <= gx < epsilon:
        return fpx/epsilon
    return (fpx*gx - fx*gpx)/(gx**2)

def h4p_p(x,epsilon,delta,sign_param,f,g,fp,gp,S,Sp):
    fx = f(x)
    gx = g(x)
    fpx = fp(x)
    gpx = gp(x)
    if - epsilon < gx < 0:
        return fpx/(-epsilon)
    elif 0 <= gx < epsilon:
        return fpx/epsilon
    return fpx/gx - fx*gpx/(gx**2)

# Universal Parameters

In [69]:
S = 1
EPSILON = [1e-4,1e-8,1e-12]
DELTA = [1e-4,1e-8,1e-12]
CASE_ZERO = [-1,1]
SIGN_PARAMS = [1]

H = [h,h1,h2,h3,h4]
HPQ = [hp_q, h1p_q, h2p_q, h3p_q, h4p_q]
HPP = [hp_p, h1p_p, h2p_p, h3p_p, h4p_p]

HEPS = [[None],EPSILON,EPSILON,EPSILON,EPSILON]
HDEL = [[None],DELTA,DELTA,[None],[None]]
HCZ = [[None]] + [CASE_ZERO]*3 + [[None]]
HNAN = [(4,"text")] + [None]*4
HPNAN = [(1,"text")] + [None]*4

HSP = [[None]] + [SIGN_PARAMS]*3 + [[None]]

YL_MOD = {'':'',1:1,2:1,3:2,4:2}
YLA = [utils.YLABEL, utils.YLABEL, utils.YLABEL_HAT, utils.YLABEL, utils.YLABEL_HAT]

gamma = utils.get_x(0,20,0.25,include_zero=True) 

# Analytic $h_i$

In [73]:
plot_h = lambda i: utils.plot(gamma,H[i],utils.call_h,h_under,(f,g),HEPS[i],HDEL[i],HCZ[i],s=S,
                              shift=-4,hindex=i,yticks=utils.get_x(0,18,6),plot_nan=HNAN[i],
                              save=os.path.join(SAVE_LOC,"h_removable"),yl_mod=YL_MOD,ylabel=YLA[i])
plot_h_black = lambda i: utils.plot(gamma,H[i],utils.call_h,h_under,(f,g),HEPS[i],HDEL[i],HCZ[i],s=S,
                              shift=-4,hindex=i,yticks=utils.get_x(0,18,6),plot_nan=HNAN[i],
                              save=os.path.join("black","h_removable"),black=1e-4,yl_mod=YL_MOD,ylabel=YLA[i])

for i in range(5):
    plot_h(i)
    if i > 0:
        plot_h_black(i)


/var/folders/4m/h0cg42rn1h97wd6m1rckrp400000gn/T/ipykernel_74815/3428388693.py:31: RuntimeWarning: invalid value encountered in double_scalars
  return f(x) / g(x)


# Analytic $h_i'$

In [74]:
plot_hp = lambda i, porq, pq: utils.plot(gamma,porq[i],utils.call_h,h_under,(f,g,fp,gp,sign,sign_p),HEPS[i],
                                         HDEL[i],HCZ[i],s=S,shift=-1,deriv=True,hindex=i,
                                         yticks=utils.get_x(0,18,6),plot_nan=HPNAN[i],
                                         save=os.path.join(SAVE_LOC,"hp_removable"),file_start=pq,yl_mod=YL_MOD,ylabel=YLA[i])
plot_hp_black = lambda i, porq, pq: utils.plot(gamma,porq[i],utils.call_h,h_under,(f,g,fp,gp,sign,sign_p),HEPS[i],
                                         HDEL[i],HCZ[i],s=S,shift=-1,deriv=True,hindex=i,
                                         yticks=utils.get_x(0,18,6),plot_nan=HPNAN[i],
                                         save=os.path.join("black","hp_removable"),file_start=pq,black=1e-4,yl_mod=YL_MOD,ylabel=YLA[i])

for i in range(5):
    plot_hp(i,HPQ,"q")
    plot_hp(i,HPP,"p")
    if i > 0:
        plot_hp_black(i,HPQ,"q")
        plot_hp_black(i,HPP,"p")

/var/folders/4m/h0cg42rn1h97wd6m1rckrp400000gn/T/ipykernel_74815/3428388693.py:39: RuntimeWarning: invalid value encountered in double_scalars
  return (fp(x)*g(x) - f(x)*gp(x))/(g(x)**2)
/var/folders/4m/h0cg42rn1h97wd6m1rckrp400000gn/T/ipykernel_74815/3428388693.py:42: RuntimeWarning: divide by zero encountered in double_scalars
  return fp(x)/g(x) - f(x)*gp(x)/(g(x)**2)
/var/folders/4m/h0cg42rn1h97wd6m1rckrp400000gn/T/ipykernel_74815/3428388693.py:42: RuntimeWarning: invalid value encountered in double_scalars
  return fp(x)/g(x) - f(x)*gp(x)/(g(x)**2)


# Pytorch and Tensorflow $h'$

In [75]:
plot_hp_pkg = lambda i,ch,pkg: utils.plot(gamma,H[i],ch,h_under,(f,g),HEPS[i],HDEL[i],HCZ[i],s=S,
                                          shift=-1,deriv=True,hindex=i,yticks=utils.get_x(0,18,6),
                                          plot_nan=HPNAN[i],
                                          save=os.path.join(SAVE_LOC,f"hp_removable_{pkg}"),yl_mod=YL_MOD,ylabel=YLA[i])

plot_hp_pkg_black = lambda i,ch,pkg: utils.plot(gamma,H[i],ch,h_under,(f,g),HEPS[i],HDEL[i],HCZ[i],s=S,
                                          shift=-1,deriv=True,hindex=i,yticks=utils.get_x(0,18,6),
                                          plot_nan=HPNAN[i],
                                          save=os.path.join("black",f"hp_removable_{pkg}"),black=1e-4,yl_mod=YL_MOD,ylabel=YLA[i])



for i in range(5):
    plot_hp_pkg(i,utils.call_h_torch,"torch")
    plot_hp_pkg(i,utils.call_h_tf,"tensorflow")
    if i > 0:
        plot_hp_pkg_black(i,utils.call_h_torch,"torch")
        plot_hp_pkg_black(i,utils.call_h_tf,"tensorflow")


# Plot Dense

In [ ]:
dg = utils.get_x(0,20,0.25,include_zero=True,sub_regions=[(7,8.25,0.01)]) 


plot_h_dense = lambda i,dg,eps,delta: utils.plot(dg,H[i],utils.call_h,h_under,(f,g),eps,delta,HCZ[i],s=S,
                              shift=-4,hindex=i,yticks=utils.get_x(0,18,6),plot_nan=HNAN[i],
                              save=os.path.join("dense_delta","h_removable"))

plot_hp_dense = lambda i, porq,pq,dg,eps,delta: utils.plot(dg,porq[i],utils.call_h,h_under,(f,g,fp,gp,sign,sign_p),eps,
                                         delta,HCZ[i],s=S,shift=-1,deriv=True,hindex=i,
                                         yticks=utils.get_x(0,18,6),plot_nan=HPNAN[i],
                                         save=os.path.join("dense_delta","hp_removable"),file_start=pq)

plot_hp_pkg_dense = lambda i,ch,pkg,dg,eps,delta: utils.plot(dg,H[i],ch,h_under,(f,g),eps,delta,HCZ[i],s=S,
                                          shift=-1,deriv=True,hindex=i,yticks=utils.get_x(0,18,6),
                                          plot_nan=HPNAN[i],
                                          save=os.path.join("dense_delta",f"hp_removable_{pkg}"))

dense_gamma = [
    utils.get_x(0,20,0.25,include_zero=True,sub_regions=[(3.9,4.1,0.001)]),
    utils.get_x(0,20,0.25,include_zero=True,sub_regions=[(7.9,8.1,0.001)]),
    utils.get_x(0,20,0.25,include_zero=True,sub_regions=[(11.9,12.1,0.001)]),
]

#plot_h_dense(1,dense_gamma[0],HEPS[i],[1e-4])


for di,d in enumerate(DELTA):
    for i in range(1,3):
        dg = dense_gamma[di]
        plot_h_dense(i,dg,HEPS[i],[d])
        plot_hp_dense(i,HPP,"p",dg,HEPS[i],[d])
        plot_hp_pkg_dense(i,utils.call_h_torch,"torch",dg,HEPS[i],[d])

for ei,e in enumerate(EPSILON):
    for i in range(3,5):
        dg = dense_gamma[ei]
        plot_h_dense(i,dg,[e],[None])
        plot_hp_dense(i,HPP,"p",dg,[e],[None])
        plot_hp_pkg_dense(i,utils.call_h_torch,"torch",dg,[e],[None])

# Verify Rounding

In [57]:
gr = utils.get_x(7,8.1,0.0000001) 
print(np.sum(np.logical_or(hp_p(gr,0,0,0,f,g,fp,gp) > 2, hp_p(gr,0,0,0,f,g,fp,gp) < 0)))
print(np.sum(np.logical_or(hp_q(gr,0,0,0,f,g,fp,gp) > 2, hp_q(gr,0,0,0,f,g,fp,gp) < 0)))

0
0


In [ ]:
import importlib
import utils
importlib.reload(utils);

#plot_h_black(3)
#plot_hp_black(3,HPP,"p")
plot_hp_pkg_black(3,utils.call_h_torch,"torch")